1. Importamos la base de datos, revisamos sus dimensiones e imprimimos los primeros 5 renglones de la misma.

In [1]:
import pandas as pd #Importamos la librería
datos=pd.read_csv("A1.4 Vino Tinto.csv") #leemos la base de datos
print("Dimensiones del data frame", datos.shape) #Revisar dimensiones con función shape
print(datos.head(5)) #Imprimir primeros 5 con función head

Dimensiones del data frame (1599, 12)
   acidezFija  acidezVolatil  acidoCitrico  azucarResidual  cloruros  \
0         7.4           0.70          0.00             1.9     0.076   
1         7.8           0.88          0.00             2.6     0.098   
2         7.8           0.76          0.04             2.3     0.092   
3        11.2           0.28          0.56             1.9     0.075   
4         7.4           0.70          0.00             1.9     0.076   

   dioxidoAzufreLibre  dioxidoAzufreTotal  densidad    pH  sulfatos  alcohol  \
0                11.0                34.0    0.9978  3.51      0.56      9.4   
1                25.0                67.0    0.9968  3.20      0.68      9.8   
2                15.0                54.0    0.9970  3.26      0.65      9.8   
3                17.0                60.0    0.9980  3.16      0.58      9.8   
4                11.0                34.0    0.9978  3.51      0.56      9.4   

   calidad  
0        5  
1        5  
2        

2. Separamos el data frame en datos de entrenamiento y datos de prueba con una proporción 80/20.

In [2]:
train=datos.sample(frac=.8, random_state=42) #Escogemos los datos de entrenamiento que sean un 80%
test = datos.drop(train.index) #Drop o soltamos los datos previamente escogidos para el entrenamiento
x_train=train.drop("calidad", axis=1) #Separamos las variables
y_train=train["calidad"] #Definimos la calidad como la variable de salida
x_test=test.drop("calidad", axis=1) #Separamos las variables
y_test=test["calidad"] #Definimos la calidad como la variable de salida
print("Train:", train.shape[0])
print("Test:",test.shape[0])
total=train.shape[0]+test.shape[0] #Verificamos si tener el total de datos
print("Total de datos: ", total)

Train: 1279
Test: 320
Total de datos:  1599


3. Generamos la metodología de selección hacia adelante, utilizando la función “SequentialFeatureSelector”, utilizando los siguientes parámetros:
   - estimator: Un modelo de regresión lineal. 
   - k_features: Se puede seleccionar la cantidad de variables de salida que se desean o un rango, en este caso de 1-8
   - forward. Determina si se hace selección hacia adelante (True)
   - scoring. La métrica que se usará para determinar si un modelo es mejor que otro, usamos R2
   - cv. Si se desea realizar validación cruzada, y cuántas instancias de la misma, 10

In [3]:
from mlxtend.feature_selection import SequentialFeatureSelector as SFS #Importamos función de la librería
from sklearn.linear_model import LinearRegression #importamos función para usar como estimator
sfs_1 = SFS(LinearRegression(), k_features=(1,8), forward=True,scoring='r2',cv=10) #Definimos los parámetros de la función de selección
sfs_1= sfs_1.fit(x_train, y_train) #Aplicamos sfs a nuestros datos de entrenamiento
print("Variables seleccionadas: ", sfs_1.k_feature_names_)

Variables seleccionadas:  ('acidezVolatil', 'cloruros', 'dioxidoAzufreLibre', 'dioxidoAzufreTotal', 'pH', 'sulfatos', 'alcohol')


4. Entrenar un modelo que solamente contenga las variables seleccionadas, predecir la respuesta en las observaciones de prueba y medir la capacidad de predicción del modelo

In [5]:
from sklearn.metrics import r2_score #Importamos función de la librería
lr=LinearRegression()
x_train_sfs = sfs_1.transform(x_train) #Quitamos variables no seleccionadas del train
x_test_sfs  = sfs_1.transform(x_test) #Quitamos variables no seleccionadas del test
lr.fit(x_train_sfs, y_train) #Ajustamos los datos de entrenamiento a la regresión lineal
y_pred = lr.predict(x_test_sfs) #Predecimos con los datos de prueba
r2 = r2_score(y_test, y_pred) #Revisamos R2 entre y predecida y y de datos de prueba 
print("R² del modelo en datos de prueba:", r2)

R² del modelo en datos de prueba: 0.42422507631408857


5. Realizar un proceso de selección hacia atrás a partir de las variables seleccionadas por el método de selección hacia adelante

In [6]:
sfs_2 = SFS(LinearRegression(), k_features=(1,5), forward=False,scoring='r2',cv=10) #Definimos los parámetros de la función de selección
sfs_2 = sfs_2.fit(x_train_sfs , y_train) #Aplicamos sfs a nuestros datos de entrenamiento ya ajustados
#Hacemos esto para que se muestren los nombres de las variables escogidas, ya que al usar SFS primero, cambia los nombres de columna a números. 
selected_first = list(sfs_1.k_feature_names_) 
idx_second = sfs_2.k_feature_idx_
selected_final = [selected_first[i] for i in idx_second] 
print("Variables seleccionadas: ", selected_final)

Variables seleccionadas:  ['acidezVolatil', 'cloruros', 'pH', 'sulfatos', 'alcohol']


6. Repetir el paso 4, pero para un modelo que contenga solamente las variables seleccionadas en el paso 5

In [7]:
x_train_sfs_2 = sfs_2.transform(x_train_sfs) #Quitamos variables no seleccionadas del train
x_test_sfs_2  = sfs_2.transform(x_test_sfs) #Quitamos variables no seleccionadas del test
lr.fit(x_train_sfs_2, y_train) #Ajustamos los datos de entrenamiento a la regresión lineal
y_pred_2 = lr.predict(x_test_sfs_2) #Predecimos con los datos de prueba
r2_back = r2_score(y_test, y_pred_2) #Revisamos R2 entre y predecida y y de datos de prueba 
print("R² del modelo en datos de prueba:", r2_back)

R² del modelo en datos de prueba: 0.4183186968674144


Aunque la R2 del paso 4 es mejor que la del paso 6, logramos eliminar 2 variables y la R2 baja solamente muy poco,